In [24]:
import pandas as pd
from datetime import datetime

pd.set_option("display.max_rows", None)

In [25]:
# 11-OCT - 31-OCT
# START_TIMESTAMP = 1728604800
# END_TIMESTAMP = 1730419200

# 08-OCT - 10-OCT
# START_TIMESTAMP = 1728345600
# END_TIMESTAMP = 1728604799

# 01 NOV - 30 NOV
# START_TIMESTAMP = 1730419201
# END_TIMESTAMP = 1732969560

# 30 NOV - 21 DEC
# START_TIMESTAMP = 1732969561
# END_TIMESTAMP = 1734782760

# 21 DIC - 18 ENE
# START_TIMESTAMP = 1734782761
# END_TIMESTAMP = 1737203944

# 18 ENE - 01 FEB
START_TIMESTAMP = 1737203945
END_TIMESTAMP = 1738412694

In [26]:
df = pd.read_csv('./data/muon_cross_TMB3-2025_01_18-2025_02_01.csv', names=['timestamp', 'TMB3'])


In [27]:
df_temp_out = pd.read_csv('./data/airport_station_hourly-2025_01_18-2025_02_01.csv')

In [12]:
# OJO, la presion esta en InHg
df_temp_in = pd.read_csv('./data/temp_Lab-2025_01_18-2025_02_01.csv', names=['timestamp', 'Temp', 'Press'])

In [13]:
df = df.sort_values('timestamp')
df.head()

,timestamp,TMB3
0,1737203945,32
1,1737203946,21
2,1737203947,29
3,1737203948,32
4,1737203949,24


In [14]:
TIME_FORMAT = "%Y-%m-%d %H:%M:%S"
def convert_string_to_timestamp(my_date: str):
    # print(my_date)
    dt_object = datetime.strptime(my_date, TIME_FORMAT)
    return int(dt_object.timestamp())

In [15]:
df_temp_out = df_temp_out.rename(columns={'time': 'datetime', 'temp': 'temp_out'})
df_temp_out.insert(1, 'timestamp', df_temp_out['datetime'].apply(convert_string_to_timestamp))

In [16]:
df_temp_out = df_temp_out[df_temp_out['timestamp'].between(START_TIMESTAMP, END_TIMESTAMP, inclusive='both')]
df_temp_out = df_temp_out.sort_values('timestamp')
df_temp_out.head()

,datetime,timestamp,temp_out,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
10,2025-01-18 10:00:00,1737205200,28.0,22.0,70.0,0.0,NaN,110.0,6.0,NaN,1008.0,NaN,2.0
11,2025-01-18 11:00:00,1737208800,31.0,22.1,59.0,0.0,NaN,90.0,9.0,NaN,1009.0,NaN,2.0
12,2025-01-18 12:00:00,1737212400,34.0,22.1,50.0,0.0,NaN,40.0,11.0,NaN,1010.0,NaN,2.0
13,2025-01-18 13:00:00,1737216000,35.0,22.0,47.0,0.0,NaN,50.0,11.0,NaN,1010.0,NaN,2.0
14,2025-01-18 14:00:00,1737219600,36.0,21.1,42.0,0.0,NaN,26.0,9.0,NaN,1009.0,NaN,2.0


In [17]:
df_temp_in['Press'] = df_temp_in['Press'].apply(lambda x: x/29.53)
df_temp_in = df_temp_in.rename(columns={'Temp': 'temp_in', 'Press': 'press_in'})
df_temp_in.head()


,timestamp,temp_in,press_in
0,1737204014,18.1,1.007111
1,1737204308,18.1,1.007111
2,1737204613,18.1,1.007111
3,1737204907,18.2,1.007111
4,1737205212,18.1,1.007111


In [18]:
merged_df = pd.merge_asof(df, df_temp_in, on='timestamp', direction='backward')
merged_df = pd.merge_asof(merged_df, df_temp_in, on='timestamp', direction='forward', suffixes=('', '_forward'))
merged_df['temp_in'] = merged_df['temp_in'].combine_first(merged_df['temp_in_forward'])
merged_df['press_in'] = merged_df['press_in'].combine_first(merged_df['press_in_forward'])
merged_df = merged_df.drop(columns=['temp_in_forward', 'press_in_forward'])


merged_df = pd.merge_asof(merged_df, df_temp_out[['timestamp', 'temp_out']], on='timestamp', direction='backward')
merged_df = pd.merge_asof(merged_df, df_temp_out[['timestamp', 'temp_out']], on='timestamp', direction='forward', suffixes=('', '_forward'))
merged_df['temp_out'] = merged_df['temp_out'].combine_first(merged_df['temp_out_forward'])
merged_df = merged_df.drop(columns=['temp_out_forward'])


In [19]:
merged_df = merged_df[['timestamp', 'temp_in', 'temp_out', 'press_in', 'TMB3']]
merged_df.head()

,timestamp,temp_in,temp_out,press_in,TMB3
0,1737203945,18.1,28.0,1.007111,32
1,1737203946,18.1,28.0,1.007111,21
2,1737203947,18.1,28.0,1.007111,29
3,1737203948,18.1,28.0,1.007111,32
4,1737203949,18.1,28.0,1.007111,24


In [20]:
import statsmodels.api as sm
import numpy as np

df = merged_df
# cols = ['inside_temp','outside_temp','outside_pressure']
cols = ['temp_in','temp_out','press_in']

mod = sm.OLS(df['TMB3'], sm.add_constant(df[cols].values))
fit = mod.fit()

errors = fit.summary2().tables[1]['Std.Err.'][1:].values
const = fit.params.values[0]
params = fit.params.values[1:]
means = df.mean()[cols].values

df['correction'] = df.apply(lambda r: np.sum((means - r[cols].values) * params), axis=1)

df['corrected'] = df['TMB3'] + df['correction']


In [21]:
df.head(100)

,timestamp,temp_in,temp_out,press_in,TMB3,correction,corrected
0,1737203945,18.1,28.0,1.007111,32,-0.048988,31.951012
1,1737203946,18.1,28.0,1.007111,21,-0.048988,20.951012
2,1737203947,18.1,28.0,1.007111,29,-0.048988,28.951012
3,1737203948,18.1,28.0,1.007111,32,-0.048988,31.951012
4,1737203949,18.1,28.0,1.007111,24,-0.048988,23.951012
5,1737203950,18.1,28.0,1.007111,28,-0.048988,27.951012
6,1737203951,18.1,28.0,1.007111,28,-0.048988,27.951012
7,1737203952,18.1,28.0,1.007111,22,-0.048988,21.951012
8,1737203953,18.1,28.0,1.007111,27,-0.048988,26.951012
9,1737203954,18.1,28.0,1.007111,29,-0.048988,28.951012


In [22]:
df[['timestamp', 'temp_in', 'temp_out', 'press_in', 'TMB3', 'corrected']].to_csv(f'./data/muon_cross_corrected_all_columns-{datetime.fromtimestamp(START_TIMESTAMP).strftime("%Y_%m_%d")}-{datetime.fromtimestamp(END_TIMESTAMP).strftime("%Y_%m_%d")}.csv', index=False)

In [23]:
df_temp_out[['timestamp', 'temp_out']].to_csv(f'./data/airport_station_hourly_processed-{datetime.fromtimestamp(START_TIMESTAMP).strftime("%Y_%m_%d")}-{datetime.fromtimestamp(END_TIMESTAMP).strftime("%Y_%m_%d")}.csv', index=False)